# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [2]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [3]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
Adapting program assessment for the age of generative AI	Conference Proceedings	edunine25	IEEE Engineering Education World Conference (EDUNINE)	2025-03-25	Virtual		"Virtual presentation of conference paper to Montevideo, Uraguay"
Hurricanes and pandemics: An experience report on adapting software engineering courses to ensure continuity of instruction	Conference Proceedings	ccsc2020	Consortium for Computing Sciences in Colleges Southeastern Conference	2021-01-22	Virtual		Virtual presentation of conference paper (Covid)
Embedding cybersecurity in the second programming course (CS2)	Conference Proceedings	ccsc2016	Consortium for Computing Sciences in Colleges Southeastern Conference	2016-11-04	Furman University		
Identifying targets for intervention by analyzing basins of attraction	Conference Proceedings	psb2011	Pacific Symposium on Biocomputing	2011-01-05	"Waimea, HI"		
Reverse Engineering and Synthesis of Biomolecular System

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [4]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Adapting program assessment for the age of gen...,Conference Proceedings,edunine25,IEEE Engineering Education World Conference (E...,2025-03-25,Virtual,NaN,Virtual presentation of conference paper to Mo...
1,Hurricanes and pandemics: An experience report...,Conference Proceedings,ccsc2020,Consortium for Computing Sciences in Colleges ...,2021-01-22,Virtual,NaN,Virtual presentation of conference paper (Covid)
2,Embedding cybersecurity in the second programm...,Conference Proceedings,ccsc2016,Consortium for Computing Sciences in Colleges ...,2016-11-04,Furman University,NaN,NaN
3,Identifying targets for intervention by analyz...,Conference Proceedings,psb2011,Pacific Symposium on Biocomputing,2011-01-05,"Waimea, HI",NaN,NaN
4,Reverse Engineering and Synthesis of Biomolecu...,Peer-Reviewed Abstract,psb2011-2,Pacific Symposium on Biocomputing,2011-01-04,"Waimea, HI",NaN,Joint presentation of session introduction wit...
5,The Impact of AI Tools on Assessment Data,Peer-Reviewed Abstract,abet2024,ABET Symposium,2024-04-04,"Tampa, FL",NaN,Oral presentation of a peer-reviewed abstract
6,Effective pedagogical practices in the compute...,Peer-Reviewed Abstract,ccsc2021,Consortium for Computing Sciences in Colleges ...,2022-01-28,"Greenville, SC",NaN,"Panel member with Professors Healy, Digh, and ..."
7,Boolean Network Models of Human Aging,Peer-Reviewed Abstract,rocky2009,7th Annual Rocky Mountains Bioinformatics Conf...,2009-12-12,"Snowmass, CO",NaN,Oral presentation of a peer-reviewed abstract
8,Creating a DevOps course,Poster,sigcse2022,ACM Technical Symposium on Computer Science Ed...,2023-03-16,"Toronto, Canada",NaN,Poster presentation of a peer-reviewed abstract
9,Template-Based Intervention in Boolean Network...,Poster,ismb2014,International Conference on Intelligent System...,2014-07-14,"Boston, MA",NaN,Poster presentation of a peer-reviewed abstract


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [5]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [6]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [7]:
!ls ../_talks

2008-12-05-rocky2008.md    2011-01-05-psb2011.md     2016-11-04-ccsc2016.md
2009-05-19-recomb2009.md   2012-03-01-talk-1.md      2021-01-22-ccsc2020.md
2009-12-12-rocky2009-2.md  2013-03-01-tutorial-1.md  2022-01-28-ccsc2021.md
2009-12-12-rocky2009.md    2014-02-01-talk-2.md      2023-03-16-sigcse2022.md
2011-01-04-psb2011-2.md    2014-03-01-talk-3.md      2024-04-04-abet2024.md
2011-01-05-psb2011-3.md    2014-07-14-ismb2014.md    2025-03-25-edunine25.md


In [9]:
!cat ../_talks/2013-03-01-tutorial-1.md

cat: ../_talks/2013-03-01-tutorial-1.md: No such file or directory
